In [1]:
import sys
import os
# Adicionando o diretório raiz na lista de diretórios para buscar módulos (p.ex: librep)
REPO_ROOT_DIR="../"
sys.path.append(os.path.dirname(REPO_ROOT_DIR))
from librepo.viewer_tool import ViewerTool
vt = ViewerTool()
vt.setup("http://vm.hiaac.ic.unicamp.br", "8081")

vt.auth('H-IAAC_Elds!')

ModuleNotFoundError: No module named 'librepo'

In [ ]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output,State
import dash_player
from plotly.subplots import make_subplots
import json


def get_list_experiment():
    experiments = vt.list_experiments()
    experiments=experiments[experiments['video']!='none']
    experiments['concat'] =experiments.index.astype(str) +'--'+experiments['experiment'] +"--"+ experiments["activity"]+"--"+ experiments["user"]+"--"+ experiments["csv_count"].astype(str)
    return dcc.Dropdown(experiments['concat'],value=experiments['concat'].iat[2], id='exp-dropdown')
    
    
def get_graph_interface():
    div_graph= html.Div( 
                         children=[
                             dcc.Dropdown([''],[], id='files-dropdown',multi=True),
                             html.Div(id='log'),
                             html.Div(id='log1'),
                             html.Div(id='click-info-output'),
                             
                             dcc.Graph(id='live-graph'), 
                             html.Label("Windows size"),
                             dcc.Input(id="windows_size-input",value=120,type="number",min=0,max=20000000,
                    # className="form-control",
                ),
                                    dcc.Graph(id='live-graph-rt'),  
                          
                         ],style={'width': '60%', 'display': 'inline-block'},
    
    
    )
              
    return div_graph

def get_video_interface():
    
    div_video= html.Div( 
                         children=[
                             dash_player.DashPlayer( id="player",url='url_video',controls=True,width="100%",height="250px",),
                             html.Div(id='number-timestamp',style={'width': '100%', 'display': 'inline-block','text-align':'center'}),
                             html.Div(id="duration-div", style={"margin": "10px 0px"},),
                             dcc.Slider(  id="intervalCurrentTime-slider", min=0,  max=1000, step=None, updatemode="drag", marks={i: str(i) for i in [0, 250, 500, 750, 1000]},value=1000,), 
                                  ],
                            style={'width': '40%', 'display': 'inline-block'},)
    
    return div_video
    

    
    
app = JupyterDash(__name__)
app.layout = html.Div(
     children=[         
         get_list_experiment(),    
         get_graph_interface(),
         get_video_interface()
             ]  
                    )


################# list experiments and url video update#
@app.callback(Output("log1", "children"),Output("player", "url"),Output("files-dropdown", "options"),Output("files-dropdown", "value"),Input('exp-dropdown', 'value'))
def update_output(value):
    import time
   
    # ts stores the time in seconds
    ts = time.time()
    print(ts)
    exp=value.split('--') 
    url =vt.get_video(exp[1], exp[2],exp[3],True)
    url1=url.split('./')
    f =vt.list_experiment_files(exp[1],exp[2],exp[3],True)
    options=f.Files#f[~f["Files"].str.contains('__')&~f["Files"].str.contains('mp4')].Files
    url='http://127.0.0.1:8888/files/noteboks/'+url1[1]
    return url,url,options,options[0:1]


@app.callback(Output("duration-div", "children"),Input("player", "duration"))
def display_duration(duration):
    return f"Duration: {duration}"

## get clickData for graph 1 
@app.callback(Output("player", "seekTo"),Input('live-graph', 'clickData'))
def display_clicked_content(click_info):
    try:
        onclic=int(click_info['points'][0]['x'])
    except:
        onclic=0.1
        
    return onclic


### update intervalcurrrentime
@app.callback(Output("player", "intervalCurrentTime"), Input("intervalCurrentTime-slider", "value"),)
def set_intervalCurrentTime(value):
    return value



# list of files an return figure
@app.callback( Output('live-graph', 'figure'), Output('log', 'children'), Input('files-dropdown', 'value'), Input('exp-dropdown', 'value'))
def create_fig(files,exp):
    exp=exp.split('--')  
    log=''
    global fig
    if(len(files)==0):
        fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    else:
        fig = make_subplots(rows=len(files), cols=1, shared_xaxes=True,subplot_titles=(files))
        row=0
        global  df_s
        df_s={}
        for f in files:
            df  = vt.get_csv(exp[1], exp[2],exp[3], f)
            df=df.sort_values(by=['VideoTimelapse'])
            df['sec'] = [x/1000 for x in df['VideoTimelapse']]
            row=row+1
            fig =vt.get_chart(df,fig,row,f.split('.')[0])
            df_s[f.split('.')[0]]=df
            
            log= " %s %s min %s max %s " % (log, f, df['sec'].min(),df['sec'].max())
            

        titles={}  
        fig.update_layout(height=int(len(files)*300),hovermode='x unified')


    
    return fig,log


@app.callback(
    Output("number-timestamp", "children"), Output('live-graph-rt', 'figure'),
    Input('files-dropdown', 'value'), Input('exp-dropdown', 'value'), Input("player", "currentTime"), Input("windows_size-input", "value"),
    
    
)
def create_fig(files,exp,timestamp,windows_size):   
   
    try:
        fig_rt = make_subplots(rows=len(files), cols=1, shared_xaxes=True,subplot_titles=(files))
        windows_size=windows_size/2
        row=1       
        for f in files:
            file=f.split('.')[0]
            df=df_s[file]
            df = df[(df['sec'] <timestamp+windows_size)&(df['sec'] >timestamp-windows_size)]
            fig =vt.get_chart(df,fig_rt,row,file)
            row=row+1
        fig_rt.add_vline(x=timestamp, line_width=3, line_dash="dash", line_color="green")
        
        fig_rt.update_layout(height=int(len(files)*300),hovermode='x unified')
       
    
    except Exception as e: 
        print(e)
        fig_rt = make_subplots(rows=1, cols=1, shared_xaxes=True)
        
    
    
    
    return timestamp,fig_rt
 
                             

In [ ]:
app.run_server(host="172.17.0.2",port=8050,  # mapped to http://tatooine:8050
    dev_tools_ui=True,
    debug=False,
    dev_tools_hot_reload=True,
    threaded=True)